In [1]:
import os
import time
import argparse

import xarray as xr
import numpy as np

from bias_correct import BiasCorrectDaily, convert_to_float32

In [2]:
# parser = argparse.ArgumentParser()
# parser.add_argument("fobserved", help="Netcdf file containing an upscaled " \
#                     "version of the observed dataset", type=str)
# parser.add_argument("fmodeled", help="Netcdf file of a GCM or Reanalysis dataset",
#                     type=str)
# parser.add_argument("var1", help="Variable name of the observed dataset")
# parser.add_argument("var2", help="Variable name of the modeled dataset")
# parser.add_argument("ofile", help="File to save bias corrected dataset")
# parser.add_argument("--njobs", help="File to save bias corrected dataset",
#                    default=1, type=int)
# args = parser.parse_args()
# args = vars(args)

In [3]:
f_observed = 'data/prism_upscaled.nc'
f_modeled = 'data/merra_filled.nc'
obs_var = 'ppt'
modeled_var = 'PRECTOTLAND'

In [4]:
obs_data = xr.open_dataset(f_observed)

In [5]:
print "loading observations"
obs_data.load()
obs_data = obs_data.dropna('time', how='all')
obs_data = obs_data.resample("D", "time")
obs_data = convert_to_float32(obs_data)

loading observations


In [6]:
print "loading modeled"
modeled_data = xr.open_dataset(f_modeled)
del modeled_data['time_bnds']
modeled_data.load()
modeled_data = modeled_data.resample("D", "time")
convert_to_float32(modeled_data)

loading modeled


<xarray.Dataset>
Dimensions:      (bnds: 2, lat: 11, lon: 9, time: 13149)
Coordinates:
  * lon          (lon) float64 -95.0 -94.38 -93.75 -93.12 -92.5 -91.88 ...
  * lat          (lat) float64 35.0 35.5 36.0 36.5 37.0 37.5 38.0 38.5 39.0 ...
  * bnds         (bnds) int64 0 1
  * time         (time) datetime64[ns] 1980-01-01 1980-01-02 1980-01-03 ...
Data variables:
    PRECTOTLAND  (time, lat, lon) float32 8.05101e-09 2.07281e-09 ...

In [7]:
print "starting bcsd"
t0 = time.time()
bc = BiasCorrectDaily(max_train_year=2001, pool=2)
corrected = bc.bias_correction(obs_data, modeled_data, obs_var,
                               modeled_var, njobs=2)
print "running time:", (time.time() - t0)
corrected.to_netcdf('merra_bc.nc')

starting bcsd
Day = 100
Running jobs 11
Day = 200
Running jobs 11
Day = 300
Running jobs 11
running time: 119.750124931


In [8]:
corrected

<xarray.Dataset>
Dimensions:         (bnds: 2, lat: 11, lon: 9, time: 12418)
Coordinates:
  * lon             (lon) float64 -95.0 -94.38 -93.75 -93.12 -92.5 -91.88 ...
  * lat             (lat) float64 35.0 35.5 36.0 36.5 37.0 37.5 38.0 38.5 ...
  * bnds            (bnds) int64 0 1
  * time            (time) datetime64[ns] 1981-01-01 1981-01-02 1981-01-03 ...
Data variables:
    bias_corrected  (time, lat, lon) float64 nan nan nan nan nan nan nan nan ...

In [9]:
obs_data

<xarray.Dataset>
Dimensions:  (lat: 11, lon: 9, time: 12418)
Coordinates:
  * lon      (lon) float64 -95.0 -94.38 -93.75 -93.12 -92.5 -91.88 -91.25 ...
  * lat      (lat) float64 35.0 35.5 36.0 36.5 37.0 37.5 38.0 38.5 39.0 39.5 ...
  * time     (time) datetime64[ns] 1981-01-01 1981-01-02 1981-01-03 ...
Data variables:
    ppt      (time, lat, lon) float32 nan nan nan nan nan nan nan nan nan ...

In [12]:
err_data=corrected.bias_corrected-obs_data.ppt

In [38]:
# Conclusion: np.mean act on xarray.DataArray with nan: remove nan, take average of the rest unmissing ones
#np.square(xr.DataArray(np.array([[np.nan,2],[3,4]])))
#np.mean([4,9,16])
np.mean(np.square(err_data))

<xarray.DataArray ()>
array(37.1043926620451)

In [68]:
import os

In [70]:
os.system("pwd")

0